In [1]:
import requests, re
import pandas as pd

In [2]:
database = []

In [3]:
url = "https://www.ndow.org/wp-json/wp/v2/event/?topic_category=322&per_page=100"

response = requests.get(url)
response

<Response [200]>

In [4]:
response.headers["X-WP-TotalPages"]

'2'

In [5]:
for i in range(0, len(response.json())):
    db = {"event_id":[], "date":[], "water_name":[], "stocked": [], "length":[]}
    
    event_id = response.json()[i]["id"]
    db["event_id"].append(event_id)
    
    date = response.json()[i]["acf"]["date"]["start"]
    db["date"].append(date)
    
    address = response.json()[i]["acf"]["address"]
    water_name = re.sub('<[^<]+?>', '', address).split(",")[0].strip()
    db["water_name"].append(water_name)
    
    excerpt = response.json()[i]["content"]["rendered"]
    
    for data in excerpt.split("\n"):
        text = re.sub('<[^<]+?>', '', data)

        if text.startswith("Stocked"):
            split_list = text.split(":")
            stocked = split_list[1]
            db["stocked"].append(stocked.strip())
            
        elif text.startswith("Length"):
            try:
                split_list = text.split(":")
                length = split_list[1]
                db["length"].append(length.strip())
            except:
                db["length"].append("Na")
    
    database.append(db)

In [29]:
df = pd.DataFrame.from_dict(database)

In [25]:
for c in df.columns:
    df[c]= df.apply(lambda x: str(x[c]).replace("[","").replace("]","").replace("'", "").replace("Inches","").replace("inches","").strip(), axis=1)
df.head(10)

event_id      date            water_name  \
0    11765  20220629         Sparks Marina   
1    11764  20220628          Spooner Lake   
2    11763  20220627           Topaz Canal   
3    11618  20220623   Wildhorse Reservoir   
4    11554  20220622     James Kinney Pond   
5    11621  20220622            Angel Lake   
6    11620  20220613            Angel Lake   
7    11567  20220622       Dacey Reservoir   
8    11553  20220621  Rose Creek Reservoir   
9    11552  20220620          Walker River   

                              stocked      length  
0                    2145 Brown Trout         9.6  
1                    1505 Tiger Trout        10.4  
2                    2016 Brown Trout         9.5  
3  249 Rainbow Trout, 2001Tiger Trout   9.3 , 9.3  
4  548 Brown Trout, 256 Rainbow Trout  9.3 , 10.2  
5                    1550 Tiger Trout         9.3  
6                   450 Rainbow Trout         8.7  
7                  4504 Rainbow Trout         9.7  
8                     201 Tiger Trout        10.8  
9                    3312 Brown Trout         9.5

In [35]:
df.apply(pd.Series.explode).reset_index()

index event_id      date                      water_name  \
0        0    11765  20220629                   Sparks Marina   
1        1    11764  20220628                    Spooner Lake   
2        2    11763  20220627                     Topaz Canal   
3        3    11618  20220623             Wildhorse Reservoir   
4        3    11618  20220623             Wildhorse Reservoir   
..     ...      ...       ...                             ...   
103     95    10078  20220506                 Mitch Park Pond   
104     96    10213  20220506             Wildhorse Reservoir   
105     97    10212  20220506             Wildhorse Reservoir   
106     98    10062  20220505               Illipah Reservoir   
107     99    11102  20220505  Truckee River at Mayberry Park   

                  stocked       length  
0        2145 Brown Trout   9.6 inches  
1        1505 Tiger Trout  10.4 Inches  
2        2016 Brown Trout   9.5 Inches  
3       249 Rainbow Trout   9.3 Inches  
4         2001Tiger Trout   9.3 Inches  
..                    ...          ...  
103     554 Rainbow Trout  10.4 Inches  
104    3069 Rainbow Trout   9.2 Inches  
105      1918 Tiger Trout   9.5 Inches  
106    5453 Rainbow Trout   9.6 Inches  
107  5000 Cutthroat Trout   8.3 Inches  

[108 rows x 6 columns]